In [7]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pmlb as dsets
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
from numpy import array as arr

# sklearn models
sys.path.append('../scores')
import scores
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import interactions
from scipy import interpolate

In [8]:
data_dir = '/scratch/users/vision/data/pmlb'
out_dir = '/scratch/users/vision/chandan/pmlb'
random_state = 42 # for each train_test_split

In [32]:
fnames = [f for f in os.listdir(out_dir) if f.startswith('single')]
results_list = [pkl.load(open(oj(out_dir, fname), "rb")) for fname in tqdm(fnames)]
# results_list = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in tqdm(fnames)]
# r = pd.concat(results_list, axis=1).T.infer_objects()


100%|██████████| 1/1 [00:00<00:00, 66.02it/s]

In [37]:
pd.concat([pd.Series(results_list[0])]).T.infer_objects()

logit_test_score                                                                     0.4925
rf_test_score                                                                        0.5125
dset_name                                 GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...
rf                                        (DecisionTreeClassifier(class_weight=None, cri...
feature_scores_mdi                        [[0.0012694066131438214, 0.0004622921889161559...
logit_score_orig_onevar                                                             [0.475]
logit_score_altered_onevar                                                          [0.485]
logit_score_altered_append                                                         [0.4925]
variances                                                                             [nan]
logit_score_altered_interaction_onevar                                              [0.485]
logit_score_altered_interaction_append                                          

In [27]:
scores_df = pd.Series({'test': [3], 'test2': [5]})
scores_df

test     [3]
test2    [5]
dtype: object

In [31]:
# row = scores_df.iloc[0]
{**row.to_dict()}

{'test': 3, 'test2': 5}

In [28]:
pd.concat([scores_df, row], axis=1)

,0,0
test,[3],3
test2,[5],5


In [ ]:
results_list = [pd.Series(pkl.load(open(oj(out_dir, fname), "rb"))) for fname in tqdm(fnames) 
                if not fname.startswith('weights') and not fname.startswith('idx')]
results = pd.concat(results_list, axis=1).T.infer_objects()

In [50]:
r = pd.DataFrame(pkl.load(open(oj(out_dir, 'full_results_7_full.pkl'), 'rb')))

In [ ]:
# plt.barh(r.dset_name, r.logit_test_score)
plt.figure(figsize=(12, 4), dpi=200)
# plt.plot(xs, r.logit_test_score)
plt.plot(r.logit_test_score, r.dset_name, '.', label='logistic train all', alpha=0.5)
plt.plot(r.rf_test_score, r.dset_name, '.', label='rf train all', alpha=0.5)
plt.plot(r.logit_score_orig_onevar, r.dset_name, '.', label='onevar orig only', alpha=0.5)
plt.plot(r.logit_score_altered_onevar, r.dset_name, '.', label='onevar altered only', alpha=0.5)
plt.plot(r.logit_score_altered_append, r.dset_name, '.', label='onevar altered append', alpha=0.5)
plt.plot(r.logit_score_altered_interaction_onevar, r.dset_name, 'x', label='one interaction only', alpha=0.5)
plt.plot(r.logit_score_altered_interaction_append, r.dset_name, 'x', label='one interaction append', alpha=0.5)
plt.ylabel('dset')
plt.xlabel('test acc')
plt.tight_layout()
plt.legend()
plt.show()
# plt.xticks(r.dset_name, rotation='vertical')

In [ ]:
plt.hist(r.logit_score_altered_interaction_append - r.logit_test_score)
plt.xlabel('interaction append - orig')
plt.show()